In [ ]:
library(class)
library(caret)
require(mlbench)
library(e1071)
library(base)
require(base)

In [2]:
data(PimaIndiansDiabetes)
data <- PimaIndiansDiabetes

In [3]:
head(data)

pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes
6,148,72,35,0,33.6,0.627,50,pos
1,85,66,29,0,26.6,0.351,31,neg
8,183,64,0,0,23.3,0.672,32,pos
1,89,66,23,94,28.1,0.167,21,neg
0,137,40,35,168,43.1,2.288,33,pos
5,116,74,0,0,25.6,0.201,30,neg


In [4]:
cat("Number of rows and coloumns:", nrow(data), ncol(data))

Number of rows and coloumns: 768 9

In [5]:
base::table(data$diabetes) 
apply(data, 2, function(x) sum(is.na(x))) 


neg pos 
500 268 

pregnant  glucose pressure  triceps  insulin     mass pedigree      age 
       0        0        0        0        0        0        0        0 
diabetes 
       0

In [6]:
SEED <- 123
set.seed(SEED)
shuffle.data <- data[base::sample(nrow(data)),]
bound <- floor(0.8*nrow(data))
df_train <- shuffle.data[1:bound,]
df_test <- shuffle.data[(bound+1):nrow(shuffle.data),]
cat("Number of training and test samples are:", nrow(df_train), nrow(df_test))

Number of training and test samples are: 614 154

In [7]:
X_train <- subset(df_train, select=-diabetes)
Y_train <- df_train$diabetes
x_test <- subset(df_test, select=-diabetes)
y_test <- df_test$diabetes

In [8]:
model_knn <- knn(train=X_train,
                 test=x_test,
                 cl=Y_train,  
                 k=3)
model_knn

[1] neg neg pos pos neg neg pos neg neg pos pos pos pos neg neg pos neg pos
 [19] neg neg neg neg pos neg pos neg neg neg neg pos neg pos neg neg neg neg
 [37] neg pos pos neg neg neg neg pos neg neg neg neg pos neg neg neg pos neg
 [55] pos neg pos neg neg neg pos pos neg pos neg pos pos neg neg neg neg pos
 [73] neg pos neg neg pos neg neg neg neg neg neg neg pos neg neg neg neg pos
 [91] pos neg neg neg neg neg neg pos neg pos neg neg neg pos pos pos neg neg
[109] pos neg pos pos neg neg neg neg neg neg neg neg neg pos pos pos pos pos
[127] neg neg neg neg neg neg neg neg neg neg neg neg neg neg pos neg neg pos
[145] neg neg pos neg neg pos neg neg neg pos
Levels: neg pos

In [9]:
conf_mat <- base::table(y_test, model_knn)
conf_mat

      model_knn
y_test neg pos
   neg  82  20
   pos  23  29

In [10]:
cat("Test accuracy: ", sum(diag(conf_mat))/sum(conf_mat))

Test accuracy:  0.7207792

In [11]:
SEED <- 2021
set.seed(SEED)
in_train <- createDataPartition(data$diabetes, p=0.7, list=FALSE)
ndf_train <- data[in_train, ]
ndf_test <- data[-in_train, ]

In [12]:
ctrl <- trainControl(method="repeatedcv", number=5, repeats=2)
nn_grid <- expand.grid(k=c(1,3,5,7))
nn_grid

k
1
3
5
7


In [13]:
set.seed(SEED)

best_knn <- train(diabetes~., data=ndf_train,
                  method="knn",
                  trControl=ctrl, 
                  preProcess = c("center", "scale"),
                  tuneGrid=nn_grid)
best_knn

k-Nearest Neighbors 

538 samples
  8 predictor
  2 classes: 'neg', 'pos' 

Pre-processing: centered (8), scaled (8) 
Resampling: Cross-Validated (5 fold, repeated 2 times) 
Summary of sample sizes: 430, 430, 431, 431, 430, 431, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa    
  1  0.6728712  0.2783917
  3  0.7174368  0.3600044
  5  0.7239356  0.3687740
  7  0.7081603  0.3290467

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 5.

In [14]:
SEED <- 123 
set.seed(SEED) 
ctrl <- trainControl(method="repeatedcv", number=5, repeats=5) 
nn_grid <- expand.grid(k=c(1, 3, 5, 7)) 
best_knn_reduced <- train(diabetes~., data=ndf_train, method="knn", 
                            trControl=ctrl, preProcess=c("center", "scale","YeoJohnson"))
X_test <- subset(ndf_test, select=-diabetes) 
pred_reduced <- predict(best_knn_reduced, newdata=X_test, model="best") 
conf_mat_best_reduced <- confusionMatrix(ndf_test$diabetes, pred_reduced) 
conf_mat_best_reduced 

Confusion Matrix and Statistics

          Reference
Prediction neg pos
       neg 130  20
       pos  44  36
                                         
               Accuracy : 0.7217         
                 95% CI : (0.659, 0.7786)
    No Information Rate : 0.7565         
    P-Value [Acc > NIR] : 0.90280        
                                         
                  Kappa : 0.3405         
                                         
 Mcnemar's Test P-Value : 0.00404        
                                         
            Sensitivity : 0.7471         
            Specificity : 0.6429         
         Pos Pred Value : 0.8667         
         Neg Pred Value : 0.4500         
             Prevalence : 0.7565         
         Detection Rate : 0.5652         
   Detection Prevalence : 0.6522         
      Balanced Accuracy : 0.6950         
                                         
       'Positive' Class : neg            
                                         